### response based on the prompt

In [1]:
from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from typing import List, Dict, Any
import requests
import numpy as np
import faiss
import os




In [2]:
import requests
import numpy as np

def generate_embeddings(text: str) :
    url = "https://api.euron.one/api/v1/euri/alpha/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer enter_api_key"
    }
    payload = {
        "input":text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    # Convert to numpy array for vector operations
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

In [3]:
from euriai import EuriaiClient


def generate_response(text):
    client = EuriaiClient(
        api_key="enter_api_key",
        model="gpt-3.5-turbo"  # You can also try: "gemini-2.0-flash-001", "llama-4-maverick", etc.
    )

    response = client.generate_completion(
        prompt=text,
        temperature=0,
        max_tokens=300
    )

    print(response)

    return response['choices'][0]['message']['content']


In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


document = TextLoader("abhishek.txt").load()
document

[Document(metadata={'source': 'abhishek.txt'}, page_content="\nMaking an Impact\nHelping Millions of Students Succeed\nAbhishek's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, factly has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Abhishek himself, came from disadvantaged backgrounds. They saw factly as a lifeline—an opportunity to rise above their circumstances.\n\nIn 2022, factly was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Abhishek remained focused on his mission. Even after the acquisition, factly continued to offer some of the most affordable and accessible tech courses in the world.\n\nThe Entrepreneur and Teacher: Abhishek's Dual Legacy\nAbhishek's journey isn't just one of entrepreneurial success; it's also a story of dedication

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.split_documents(document)

In [6]:
len(docs)

58

In [7]:
[len(doc.page_content) for doc in docs]


[53,
 295,
 163,
 291,
 52,
 288,
 132,
 270,
 140,
 125,
 297,
 141,
 293,
 157,
 296,
 140,
 298,
 221,
 295,
 117,
 297,
 82,
 257,
 293,
 296,
 289,
 53,
 299,
 104,
 6,
 38,
 292,
 61,
 292,
 278,
 246,
 177,
 283,
 110,
 290,
 276,
 45,
 292,
 274,
 280,
 215,
 294,
 256,
 287,
 269,
 268,
 298,
 282,
 298,
 299,
 283,
 210,
 297]

In [9]:
docs[0].page_content

'Making an Impact\nHelping Millions of Students Succeed'

In [10]:
len(docs[0].page_content)

53

In [11]:
docs[0].page_content


'Making an Impact\nHelping Millions of Students Succeed'

In [13]:
texts = [doc.page_content for doc in docs]
texts

['Making an Impact\nHelping Millions of Students Succeed',
 "Abhishek's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, factly has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,",
 'Many of these students, like Abhishek himself, came from disadvantaged backgrounds. They saw factly as a lifeline—an opportunity to rise above their circumstances.',
 'In 2022, factly was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Abhishek remained focused on his mission. Even after the acquisition, factly continued to offer some of the most affordable and accessible tech courses in the world.',
 "The Entrepreneur and Teacher: Abhishek's Dual Legacy",
 "Abhishek's journey isn't just one of entrepreneurial success; it's also a story of dedication to teaching. Throughout 

### converting into embeddings

In [15]:
embeddings = [generate_embeddings(i) for i in texts]
embeddings

[array([ 0.03395454, -0.0469909 ,  0.03878181, ..., -0.02398636,
        -0.01851818,  0.01633636]),
 array([ 0.00222121,  0.01417504,  0.04687936, ...,  0.00848468,
        -0.00606339,  0.00479171]),
 array([ 0.00835684,  0.00321195,  0.0240378 , ...,  0.00796278,
        -0.03285665,  0.00099195]),
 array([-0.03912006, -0.03953902,  0.01851264, ..., -0.0165095 ,
         0.00166028,  0.03859637]),
 array([ 0.0497135 ,  0.00248864,  0.04778724, ..., -0.03979191,
         0.02365613, -0.0001878 ]),
 array([ 0.02293671,  0.00282336,  0.05224019, ..., -0.02238865,
         0.00464098,  0.00543391]),
 array([ 0.01883637, -0.00435591,  0.04397568, ...,  0.00189118,
         0.01190555,  0.01604714]),
 array([ 0.04013287, -0.037623  ,  0.05436385, ...,  0.00638449,
        -0.00923633,  0.0197025 ]),
 array([ 0.00418858, -0.02390262,  0.00359171, ...,  0.01201451,
        -0.01445111, -0.03555901]),
 array([ 0.0192949 , -0.00051076,  0.05972037, ..., -0.0209674 ,
         0.00802255, -0.02

In [16]:
generate_embeddings("my name is abhishek")


array([ 0.01690955, -0.02679409, -0.00042851, ...,  0.0048325 ,
        0.02119885,  0.00627635])

In [17]:
generate_embeddings("This is a test input.")


array([ 0.02033959,  0.0112574 ,  0.01501457, ...,  0.0141459 ,
       -0.01669541, -0.00123944])

### creating emneddings

In [19]:
embeddings  = []
valid_texts = []
for i ,text in enumerate(texts[0:8]):
    clean_text = text.strip()
    if not clean_text:
        print(f"Skipping empty text at index {i}")
        continue
    embedding = generate_embeddings(text)
    embeddings.append(embedding)
    valid_texts.append(text)
    print(f"Processed {i+1}/{len(texts)}: {text[:30]}... -> {embedding[:5]}")  # Print first 30 characters of text and first 5 elements of embedding

Processed 1/58: Making an Impact
Helping Milli... -> [ 0.03394562 -0.04700582  0.03882615  0.07890651  0.02693837]
Processed 2/58: Abhishek's commitment to affor... -> [0.00222121 0.01417504 0.04687936 0.03155133 0.02719708]
Processed 3/58: Many of these students, like A... -> [0.00837254 0.00321276 0.0240167  0.06420763 0.05594382]
Processed 4/58: In 2022, factly was acquired b... -> [-0.03912006 -0.03953902  0.01851264  0.01957312  0.03432825]
Processed 5/58: The Entrepreneur and Teacher: ... -> [0.0497135  0.00248864 0.04778724 0.02062161 0.02595182]
Processed 6/58: Abhishek's journey isn't just ... -> [0.02293671 0.00282336 0.05224019 0.03950664 0.02735613]
Processed 7/58: in Big Data, Data Science, or ... -> [ 0.01872769 -0.00426837  0.04363758 -0.01171236  0.06529939]
Processed 8/58: His commitment to affordable e... -> [ 0.04013287 -0.037623    0.05436385  0.05797807  0.02861255]


In [20]:
embeddings

[array([ 0.03394562, -0.04700582,  0.03882615, ..., -0.02400732,
        -0.01851331,  0.0163457 ]),
 array([ 0.00222121,  0.01417504,  0.04687936, ...,  0.00848468,
        -0.00606339,  0.00479171]),
 array([ 0.00837254,  0.00321276,  0.0240167 , ...,  0.00799877,
        -0.03286495,  0.00101854]),
 array([-0.03912006, -0.03953902,  0.01851264, ..., -0.0165095 ,
         0.00166028,  0.03859637]),
 array([ 0.0497135 ,  0.00248864,  0.04778724, ..., -0.03979191,
         0.02365613, -0.0001878 ]),
 array([ 0.02293671,  0.00282336,  0.05224019, ..., -0.02238865,
         0.00464098,  0.00543391]),
 array([ 0.01872769, -0.00426837,  0.04363758, ...,  0.00197872,
         0.01199611,  0.01609543]),
 array([ 0.04013287, -0.037623  ,  0.05436385, ...,  0.00638449,
        -0.00923633,  0.0197025 ])]

In [21]:
embeddings[0]

array([ 0.03394562, -0.04700582,  0.03882615, ..., -0.02400732,
       -0.01851331,  0.0163457 ])

In [22]:
import time
import numpy as np

embeddings = []

for i, text in enumerate(texts):
    try:
        print(f"🔄 Processing {i+1}/{len(texts)}...")
        embedding = generate_embeddings(text).astype('float32')
        embeddings.append(embedding)
        time.sleep(1)  # ⏳ Add delay to respect API rate limits
    except ValueError as e:
        print(f"❌ Error on text {i+1}: {e}")


🔄 Processing 1/58...
🔄 Processing 2/58...
🔄 Processing 3/58...
🔄 Processing 4/58...
🔄 Processing 5/58...
🔄 Processing 6/58...
🔄 Processing 7/58...
🔄 Processing 8/58...
🔄 Processing 9/58...
🔄 Processing 10/58...
🔄 Processing 11/58...
🔄 Processing 12/58...
🔄 Processing 13/58...
🔄 Processing 14/58...
🔄 Processing 15/58...
🔄 Processing 16/58...
🔄 Processing 17/58...
🔄 Processing 18/58...
🔄 Processing 19/58...
🔄 Processing 20/58...
🔄 Processing 21/58...
🔄 Processing 22/58...
🔄 Processing 23/58...
🔄 Processing 24/58...
🔄 Processing 25/58...
🔄 Processing 26/58...
🔄 Processing 27/58...
🔄 Processing 28/58...
🔄 Processing 29/58...
🔄 Processing 30/58...
🔄 Processing 31/58...
🔄 Processing 32/58...
🔄 Processing 33/58...
🔄 Processing 34/58...
🔄 Processing 35/58...
🔄 Processing 36/58...
🔄 Processing 37/58...
🔄 Processing 38/58...
🔄 Processing 39/58...
🔄 Processing 40/58...
🔄 Processing 41/58...
🔄 Processing 42/58...
🔄 Processing 43/58...
🔄 Processing 44/58...
🔄 Processing 45/58...
🔄 Processing 46/58.

In [23]:
docs[0:10]

[Document(metadata={'source': 'abhishek.txt'}, page_content='Making an Impact\nHelping Millions of Students Succeed'),
 Document(metadata={'source': 'abhishek.txt'}, page_content="Abhishek's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, factly has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,"),
 Document(metadata={'source': 'abhishek.txt'}, page_content='Many of these students, like Abhishek himself, came from disadvantaged backgrounds. They saw factly as a lifeline—an opportunity to rise above their circumstances.'),
 Document(metadata={'source': 'abhishek.txt'}, page_content='In 2022, factly was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Abhishek remained focused on his mission. Even after the acquisition, factly continued to offer some of th

In [24]:
embeddings[0].shape

(1536,)

### storing embeddings into vector database

In [25]:
import faiss
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

In [26]:
dimension = embeddings[0].shape[0]
faiss_index = faiss.IndexFlatL2(dimension)

faiss_index.add(np.array(embeddings).astype('float32'))

In [27]:
generate_response("tell me about abhishek")

{'id': 'chatcmpl-BhDEmAzT2SL4UTL7W5TuKRxiZarTt', 'object': 'chat.completion', 'created': 1749638688, 'model': 'gpt-3.5-turbo', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Abhishek is a common Indian male given name that means "fearless" or "without fear". It is also a popular name in Hindu mythology, where it is associated with various gods and heroes. Abhishek is often used as a first name for boys in India, and is sometimes shortened to "Abhi" or "Abhay". People named Abhishek are generally believed to be courageous, confident, and determined individuals.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 89, 'total_tokens': 102}}


'Abhishek is a common Indian male given name that means "fearless" or "without fear". It is also a popular name in Hindu mythology, where it is associated with various gods and heroes. Abhishek is often used as a first name for boys in India, and is sometimes shortened to "Abhi" or "Abhay". People named Abhishek are generally believed to be courageous, confident, and determined individuals.'

In [30]:
document = [Document(page_content=text) for text in texts[:len(embeddings)]]

In [31]:
document

[Document(metadata={}, page_content="Making an Impact\nHelping Millions of Students Succeed\nAbhishek's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, factly has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Abhishek himself, came from disadvantaged backgrounds. They saw factly as a lifeline—an opportunity to rise above their circumstances.\n\nIn 2022, factly was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Abhishek remained focused on his mission. Even after the acquisition, factly continued to offer some of the most affordable and accessible tech courses in the world."),
 Document(metadata={}, page_content="The Entrepreneur and Teacher: Abhishek's Dual Legacy\nAbhishek's journey isn't just one of entrepreneurial success; it's also a story of

In [32]:
embeddings

[array([-0.00385632, -0.03016803,  0.02152714, ..., -0.00815629,
         0.0045252 ,  0.02842074]),
 array([ 0.04461834, -0.01665454,  0.04145075, ..., -0.01026989,
         0.00267574,  0.00132163]),
 array([ 0.02154558, -0.01480704,  0.04501645, ..., -0.00432875,
         0.00943648,  0.00042551])]

In [30]:
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
document = [Document(page_content=text) for text in texts[:len(embeddings)]]

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

faiss_index = faiss.IndexFlatL2(dimension)
vector_store = FAISS(
    embedding_function=generate_embeddings,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
vector_store.add_documents(document)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['f77a4e59-6dd6-461e-9727-53d73be3956a',
 'c5353bce-7e47-4549-91a9-a6e1f4ba0ac3',
 '9e36ac8a-e5f6-4c85-a089-b9dcd25e943e',
 'd2c1652f-cc9a-4107-975e-d13145930f1f',
 '97fea5e2-769c-4a72-8258-71b6bb965087',
 '5c1dbb15-77f6-41d8-a3e4-9dd0495ad338',
 '0b80270e-3678-401a-9fce-06598ccb70ae',
 'f07506c7-c9fd-4885-b1f3-cd0e8d17f19f',
 '200ec2f1-3dfb-40fe-8c3d-28af86259545',
 '15774c96-7072-48d4-9c1d-3aad50681558',
 '70b1cdc1-82e7-4060-8c66-f9370c868bc9',
 '101593e9-bd6e-49c6-9ac2-21b396743e79',
 'ca4f4d6e-4d47-4674-b196-2f1f1a601eaf',
 'c81b3a4d-53c2-4cb1-a505-0d3c1ea43eef',
 'daf26471-a6cf-4e97-9cf1-d1e358fbdb98',
 '78779eaf-417d-479b-9d74-f96e2f39d104',
 '4abe55a1-5afa-4351-8e20-0819a2964e39',
 'a8b35419-4ad3-4836-9313-895a41221e8c',
 '039dc675-5ace-4e2d-bc02-6b7eb8c0f9fe',
 'b785baad-7f28-4b08-bd73-cd590c01133a',
 'e3e4fe58-dece-4296-be1d-07f09ee1d222',
 'd28cfc4c-0ca5-4372-8a88-6dde6bcd3619',
 'a446682d-0d16-40f7-a37f-fc924cf23159',
 'cd64e2d3-ee7e-44c8-a2b9-8cd77ebc50df',
 '8f1a4ba0-7e56-

In [31]:
vector_store.save_local("faiss_index")

In [32]:
retriver = vector_store.as_retriever(search_kwargs={"k": 2})

### real use case of langchain


In [ ]:
from langchain.chains import RetrievalQA


RetrievalQA.from_chain_type(llm,retrival)


In [34]:
from euriai import EuriaiLangChainLLM

llm = EuriaiLangChainLLM(
    api_key="enter_api_key",
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=300
)

print(llm.invoke("Write a poem about time travel."))

In the depths of night, a shimmering portal appears
A gateway to the past, erasing all fears
Time travel beckons, a journey through the ages
A chance to rewrite history's ever-turning pages

Step through the portal, feel the rush of the wind
As you're whisked away to a time long rescinded
The world around you changes, morphing and shifting
As you navigate the currents of time, ever drifting

Back to the days of knights and chivalry
Or forward to a future of technology
Time travel grants you the power to explore
The mysteries of the past, the unknown of before

But beware, traveler, for time is a fickle friend
Changing the course of history with every bend
The consequences of your actions, unknown and vast
As you navigate the currents of time, ever fast

So tread carefully, with reverence and care
For time travel is a gift, rare and rare
A chance to glimpse the world as it once was
And marvel at the wonders of time's ever-circling buzz

Embrace the journey, embrace the unknown
For in th

In [35]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = retriver)

In [36]:
rag_chain("who is abhishek")

/var/folders/dp/w9766y9x00j7vmxb8b37p1bc0000gp/T/ipykernel_82717/748377218.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_chain("who is abhishek")


{'query': 'who is abhishek',
 'result': 'Abhishek Kumar is a person who was born in Jamshedpur, Jharkhand, India, to a family of modest means and faced financial hardship in his early years. He eventually excelled academically and earned a degree in Computer Science and Engineering.'}

In [37]:
rag_chain("who is arun")

{'query': 'who is arun', 'result': "I don't know who Arun is."}

In [39]:
generate_response("who is abhishek")

{'id': 'chatcmpl-BhDSmK3m6N27HmSPXH8LWjwOQwh9M', 'object': 'chat.completion', 'created': 1749639556, 'model': 'gpt-3.5-turbo', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Abhishek is a common Indian given name for males. It means "divine" or "the source of light" in Sanskrit. It is also the name of a popular Indian actor, Abhishek Bachchan, who works in Hindi films.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 12, 'completion_tokens': 52, 'total_tokens': 64}}


'Abhishek is a common Indian given name for males. It means "divine" or "the source of light" in Sanskrit. It is also the name of a popular Indian actor, Abhishek Bachchan, who works in Hindi films.'

In [40]:
generate_response("who is arun")

{'id': 'chatcmpl-BhDT0IeLYHlIA0Ohbu6BE2N5853eh', 'object': 'chat.completion', 'created': 1749639570, 'model': 'gpt-3.5-turbo', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'There are many individuals named Arun in the world, so it is difficult to determine which specific person you are referring to without more context. Arun is a common Indian name that means "sun" in Sanskrit.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 11, 'completion_tokens': 44, 'total_tokens': 55}}


'There are many individuals named Arun in the world, so it is difficult to determine which specific person you are referring to without more context. Arun is a common Indian name that means "sun" in Sanskrit.'

In [41]:
rag_chain("give me the comapny whre abhishek is working")

{'query': 'give me the comapny whre abhishek is working',
 'result': 'Abhishek has worked with companies like Wipro, Deloitte, Verizon Labs, and Ernst & Young.'}

In [42]:
rag_chain("what is the name of the company whre abhishek is working")

{'query': 'what is the name of the company whre abhishek is working',
 'result': 'Abhishek has worked at companies like Wipro, Deloitte, Verizon Labs, and Ernst & Young.'}

In [43]:
rag_chain("currently where abhishek is working")

{'query': 'currently where abhishek is working', 'result': 'Ernst & Young'}